In [1]:
# this script reads an input file (csv) with the following columns: gene, predicted_label, confidence, description, annotated
# it also reads a list of genes (one per line) from a csv file with a header "gene"
# the first csv file contains a list of genes and their GO terms annotations in the column "predicted_label"
# First filter the file to keep annotated == "Yes"
# Second, take the list of genes from the second file and calculate an enrichment test for the list of GO terms given in the first file
# The enrichment test is done using the hypergeometric test from scipy.stats.hypergeom
# The output is a csv file with the following columns: predicted_label, pvalue, enrichment
# use pandas to read the csv files
# import the files in the terminal line using the following command:
# python proteinfer_enrich.py input.csv genes.csv

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import hypergeom
from tqdm import tqdm
import multiprocessing as mp